<a href="https://colab.research.google.com/github/pras-4795/JPMCColab/blob/main/MLE_CleanCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_openml
import ssl
import certifi

ssl._create_default_https_context = ssl._create_default_https_context
dataset = fetch_openml(data_id=40945, as_frame=True)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [ ]:
df = dataset.frame
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,None,"St Louis, MO"
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

def ohe(df, col):
  le = LabelEncoder()
  a = le.fit_transform(df[col]).reshape(-1,1)
  columns = [f'{col}_{name}'for name in le.classes_]
  encoder = OneHotEncoder(sparse=False, categories='auto')
  result = pd.DataFrame(encoder.fit_transform(a),columns=columns)
  return result

In [ ]:
class NumInPartyTransformer(BaseEstimator, TransformerMixin):

  def __init__(self):
    super().__init__()

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    X['num_in_party'] = X['parch'] + X['sibsp']
    X['num_in_party'] = np.where(X['num_in_party'].isin([1]), 'Alone',
                        np.where(X['num_in_party'].isin([2,3,4]), 'Small',
                        np.where(X['num_in_party'].isin([5,6,7]), 'Medium', 'Large')))
    result = ohe(X,'num_in_party')
    return pd.concat([X, result], axis=1).drop(['parch', 'sibsp', 'num_in_party'], axis=1)


In [ ]:
class AgeTransformer(BaseEstimator, TransformerMixin):

  def __init__(self, buckets=10):
    super().__init__()
    self.__buckets = buckets

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    X['age'] = X.groupby(['sex', 'pclass'])['age'].apply(lambda val : val.fillna(val.mean()))

    age_labels = [f'age_{x}' for x in range(self.__buckets)]
    X['age_bucket'] = pd.qcut(X['age'], self.__buckets, labels=age_labels)
    result = ohe(X,'age_bucket')
    return pd.concat([X, result], axis=1).drop(['age', 'age_bucket'], axis=1)

In [ ]:
class CabinTransformer(BaseEstimator, TransformerMixin):

  def __init__(self, buckets=10):
    super().__init__()
    self.__buckets = buckets

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    X['deck'] = X['cabin'].apply(lambda cabin : cabin[0] if pd.notnull(cabin) else 'X')
    deck_encoder = LabelEncoder()
    X['deck'] = deck_encoder.fit_transform(X['deck'])

    return X.drop(['cabin'], axis=1)

In [ ]:
class ColumnDropper(BaseEstimator, TransformerMixin):

  def __init__(self, columns):
    super().__init__()
    self.__columns = columns

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    return X.drop(self.__columns, axis=1)

In [ ]:
class FareTransformer(BaseEstimator, TransformerMixin):

  def __init__(self, buckets=10):
    super().__init__()
    self.__buckets = buckets

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    X['fare'] = X['fare'].fillna(0)
    fare_labels = [f'fare_{i}' for i in range(self.__buckets)]
    X['fare_bucket'] = pd.qcut(X['fare'], self.__buckets, labels=fare_labels)
    fare_buckets = ohe(X, 'fare_bucket')
    return pd.concat([X, fare_buckets], axis=1).drop(['fare', 'fare_bucket'], axis=1)

In [ ]:
class EmabrkedTransformer(BaseEstimator, TransformerMixin):

  def __init__(self, buckets=10):
    super().__init__()
    self.__buckets = buckets

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    X['embarked'] = X['embarked'].fillna('S').astype('object')
    embarked_encoder = LabelEncoder()
    X['embarked'] = embarked_encoder.fit_transform(X['embarked'])
    return X

In [ ]:
class SexTransformer(BaseEstimator, TransformerMixin):

  def __init__(self, buckets=10):
    super().__init__()
    self.__buckets = buckets

  def fit(self, X, Y=None):
    return self

  def transform(self, X):
    sex_encoder = LabelEncoder()
    X['sex'] = sex_encoder.fit_transform(X['sex'])
    return X

In [ ]:
data_pipe = Pipeline([
    ('num_in_party', NumInPartyTransformer()),
    ('age', AgeTransformer(7))
    ('cleanup', ColumnDropper(columns=['name','ticket','boat','home.dest', 'body', 'survived']))
])

In [ ]:
df2 = data_pipe.fit_transform(df)
df2.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
<ipython-input-10-68336c6291b1>:11: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  X['age'] = X.groupby(['sex', 'pclass'])['age'].apply(lambda val : val.fillna(val.mean()))
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored u

,pclass,survived,name,sex,ticket,fare,cabin,embarked,boat,body,...,num_in_party_Medium,num_in_party_Small,age_bucket_age_0,age_bucket_age_1,age_bucket_age_2,age_bucket_age_3,age_bucket_age_4,age_bucket_age_5,age_bucket_age_6,age_bucket_nan
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,24160,211.3375,B5,S,2,None,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1,"Allison, Master. Hudson Trevor",male,113781,151.5500,C22 C26,S,11,NaN,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0,"Allison, Miss. Helen Loraine",female,113781,151.5500,C22 C26,S,None,NaN,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,113781,151.5500,C22 C26,S,None,135.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,113781,151.5500,C22 C26,S,None,NaN,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


<h1>Train and Store the model</h1>

In [ ]:
import pickle
from sklearn.linear_model import LogisticRegression

features = df2.copy()
labels = df['survived']

model = LogisticRegression()
model.fit(features, labels)

In [ ]:
with open('model.pic', 'wb') as out_file:
  pickle.dump(model, out_file)
  pickle.dump(data_pipe, out_file)

In [ ]:
with open('model.pic', 'rb') as in_file:
  model = pickle.load(in_file)
  data_pipe = pickle.load(in_file)

In [ ]:
df2 = data_pipe.fit_transform(df)
result = model.predict(df2)

In [ ]:
result